In [ ]:
import numpy as np
from src.rowwise_metrics import direct_mutual_information, euclid, cosine
from src.simulation import get_spectrum

In [ ]:
PARAMS = dict(
    Te=1.0,
    Ne=10**18,
    resolution=10000,
    low_w=200,
    upper_w=1000,
    max_ion_charge=3,
    webscraping='dynamic',
)

n = 20
elements = sorted(['Fe', 'Al']) # WARNING! sorts the elements in alphabetical order to prevent duplicities
path = f'data/{elements}_{n}.npy'
try:
    spectra = np.load(open(path, 'rb'), allow_pickle=True) # TODO allow pickle ?
except OSError:
    if not input("Data not found. Input empty string to dowload the data."):
        spectra = np.stack(
            list(map(
            lambda x: get_spectrum(
                elements=elements,
                percentages=[x,100-x],
                **PARAMS,
            ),
            np.linspace(0, 100, n), 
            )),
            axis=0,
        )
        np.save(open(path, 'wb'), spectra)

In [ ]:
metrics = [euclid, cosine, direct_mutual_information]
matrices = [np.zeros((n, n)) for _ in metrics]
for i in range(len(metrics)):
    for j in range(1, n-1):
        for k in range(1, n-1):
            matrices[i][j][k] = metrics[i](spectra[j] / np.sum(spectra[j]), spectra[k] / np.sum(spectra[k]))

In [ ]:
import plotly.express as px
px.imshow(matrices[0])

In [ ]:
px.imshow(matrices[1])

In [ ]:
px.imshow(matrices[2])

In [ ]:
from libs_tools.visualization.spectral import plot_spectra

matrices[0] = matrices[0] / matrices[0][0].max()
matrices[2] = matrices[2] - matrices[2][0].min()
plot_spectra([matrix[0] for matrix in matrices], labels=[foo.__name__ for foo in metrics])